In [38]:
import nltk
from nltk.corpus import brown
import re
import numpy as np

In [4]:
QUESTIONS_PATH = '/home/jack/NLP/questions.txt'

In [91]:
brown_words = brown.words()
unigram_count = nltk.FreqDist(brown_words)
number_of_words = len(unigram_count)
bigram_count = nltk.FreqDist(nltk.bigrams(brown_words))

In [35]:
questions = []

with open(QUESTIONS_PATH, 'r') as f:
    for line in f:
        split = line.split(':')
        sentence = re.findall('\w+', split[0])
        answers = re.findall('\w+', split[1])
        questions.append((line, sentence, answers))

In [93]:
def unigram_homophone_selector(questions):
    for question in questions:
        line = question[0].split(':')[0].strip('\n')
        homophones = question[2]
        
        index = np.argmax([unigram_count[homophone] for homophone in homophones])
        best_homophone = homophones[index]
        
        print line + ' -> ' + best_homophone

In [88]:
def bigram_probability(first, second):
    if unigram_count[first] == 0:
        return 0.0
    else:
        return bigram_count[(first, second)] / float(unigram_count[first])

In [110]:
def smoothed_bigram_probability(first, second):
    return (bigram_count[(first, second)] + 1) / (float(unigram_count[first]) + number_of_words)

In [108]:
def bigram_homophone_selector(questions):
    for question in questions:
        line = question[0].split(':')[0].strip('\n')
        homophones = question[2]
        sentence = question[1]
        
        best_probablitiy = 0
        best_homophone = 'NOT_FOUND'
        for homophone in homophones:
        
            word_position = sentence.index('____')
        
            if word_position == 0:
                after = sentence[word_position + 1]
                probabilility = bigram_probability(homophone, after)
            elif word_position == len(sentence) - 1:
                before = sentence[word_position - 1]
                probabilility = bigram_probability(before, homophone)
            else:
                before = sentence[word_position - 1]
                after = sentence[word_position + 1]
                probabilility = bigram_probability(before, homophone) * bigram_probability(homophone, after)
            
            if probabilility > best_probablitiy:
                best_homophone = homophone
                best_probablitiy = probabilility
    
        print line + ' -> ' + best_homophone

In [120]:
def smoothed_bigram_homophone_selector(questions):
    for question in questions:
        line = question[0].split(':')[0].strip('\n')
        homophones = question[2]
        sentence = question[1]
        
        best_probablitiy = None
        best_homophone = 'NOT_FOUND'
        for homophone in homophones:
        
            word_position = sentence.index('____')
        
            if word_position == 0:
                after = sentence[word_position + 1]
                probabilility = np.log10(smoothed_bigram_probability(homophone, after))
            elif word_position == len(sentence) - 1:
                before = sentence[word_position - 1]
                probabilility = np.log10(smoothed_bigram_probability(before, homophone))
            else:
                before = sentence[word_position - 1]
                after = sentence[word_position + 1]
                probabilility = np.log10(smoothed_bigram_probability(before, homophone)) + np.log10(smoothed_bigram_probability(homophone, after))
            
            if not best_probablitiy:
                best_homophone = homophone
                best_probablitiy = probabilility
            if probabilility > best_probablitiy:
                best_homophone = homophone
                best_probablitiy = probabilility
    
        print line + ' -> ' + best_homophone

In [121]:
print '-' * 10 + 'Unigram Selector' + '-' * 10
unigram_homophone_selector(questions)
print ''
print '-' * 10 + 'Bigram Selector' + '-' * 10
bigram_homophone_selector(questions)
print ''
print '-' * 10 + 'Smoothed Bigram Selector' + '-' * 10
smoothed_bigram_homophone_selector(questions)

----------Unigram Selector----------
I don't know ____ to go out or not .  -> whether
Houses were being built on this ____ .  -> sight
We went ____ the door to get inside .  -> through
I really want a ____ car .  -> new
They all had a ____ of the cake .  -> peace
She had to go to ____ to prove she was innocent .  -> court
We were only ____ to visit at certain times .  -> allowed
We had to ____ a car while we were on holiday .  -> higher
Tip the jug and ____ lots of cream on the strawberries .  -> poor
She went back to ____ she had locked the door .  -> check

----------Bigram Selector----------
I don't know ____ to go out or not .  -> whether
Houses were being built on this ____ .  -> site
We went ____ the door to get inside .  -> through
I really want a ____ car .  -> new
They all had a ____ of the cake .  -> piece
She had to go to ____ to prove she was innocent .  -> court
We were only ____ to visit at certain times .  -> NOT_FOUND
We had to ____ a car while we were on holiday .  -> 